# Recurrent Neural Networks

## 1. 引言

在传统的神经网络（如多层感知机MLP或卷积神经网络CNN）中，每次输入都是独立的，模型无法记住之前的输入信息。然而，在许多现实世界的任务中，数据是序列化的，例如文本、语音、视频等，其中当前的信息与之前的信息高度相关。为了处理这类具有“记忆”需求的数据，循环神经网络（RNN）应运而生。

## 2. 基本循环神经网络 (Basic RNN)

基本RNN的核心思想是：它对序列的每个元素都执行相同的操作，并且每次操作的输出都依赖于前一次计算的结果。这使得RNN能够捕获序列中的时间依赖性。

### 2.1 架构与参数共享

RNN通过一个“隐藏状态”（Hidden State，通常表示为 $h$）来传递信息。在每个时间步 $t$，RNN接收当前输入 $x_t$ 和前一个时间步的隐藏状态 $h_{t-1}$，然后计算新的隐藏状态 $h_t$ 和当前输出 $y_t$。

模型中，执行转换的函数 $f$（通常包含权重矩阵 $W_{hh}$, $W_{xh}$, $W_{hy}$ 和偏置项）在所有时间步都是**共享的**。这意味着无论输入/输出序列有多长，我们只需要学习一组参数。

*   **输入:** $x^1, x^2, x^3, \dots$
*   **隐藏状态:** $h^0, h^1, h^2, h^3, \dots$ ($h^0$ 通常是初始状态，可以是零向量)
*   **输出:** $y^1, y^2, y^3, \dots$
*   **核心函数:** $f$ (在所有时间步共享)

### 2.2 数学表达式 (Naïve RNN)

假设函数 $f$ 定义为：$h', y = f(h, x)$

1.  **隐藏状态更新：**
    $h_t = \sigma(W_{hh} h_{t-1} + W_{xh} x_t + b_h)$
    其中，$\sigma$ 是激活函数（如ReLU, Tanh），$W_{hh}$ 是连接前一隐藏状态的权重矩阵，$W_{xh}$ 是连接当前输入的权重矩阵，$b_h$ 是隐藏状态的偏置项。

2.  **输出计算：**
    $y_t = \sigma(W_{hy} h_t + b_y)$
    其中，$W_{hy}$ 是连接当前隐藏状态到输出的权重矩阵，$b_y$ 是输出的偏置项。


In [1]:
import torch
import torch.nn as nn

# 1. 基本RNN的前向传播 (手动实现简化版)
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        
        # 定义权重矩阵和偏置
        self.W_xh = nn.Linear(input_size, hidden_size, bias=True) # W_i, bias_i
        self.W_hh = nn.Linear(hidden_size, hidden_size, bias=True) # W_h, bias_h
        self.W_hy = nn.Linear(hidden_size, output_size, bias=True) # W_o, bias_o
        
        self.activation = nn.Tanh() # 激活函数，PPT中用sigma表示

    def forward(self, input_seq):
        # input_seq: (seq_len, batch_size, input_size)
        
        # 初始化隐藏状态 h0
        # h0: (batch_size, hidden_size)
        batch_size = input_seq.size(1)
        hidden_state = torch.zeros(batch_size, self.hidden_size).to(input_seq.device)
        
        outputs = []
        for i in range(input_seq.size(0)): # 遍历序列中的每个时间步
            current_input = input_seq[i] # current_input: (batch_size, input_size)
            
            # 计算新的隐藏状态
            # h_t = activation(W_xh * x_t + W_hh * h_{t-1} + b_h)
            combined_input = self.W_xh(current_input) + self.W_hh(hidden_state)
            hidden_state = self.activation(combined_input)
            
            # 计算当前输出
            # y_t = activation(W_hy * h_t + b_y)
            output = self.W_hy(hidden_state)
            outputs.append(output)
            
        # outputs: list of (batch_size, output_size) -> concatenate to (seq_len, batch_size, output_size)
        return torch.stack(outputs, dim=0), hidden_state

# 示例使用
input_size = 10
hidden_size = 20
output_size = 5
seq_len = 3
batch_size = 4

model = SimpleRNN(input_size, hidden_size, output_size)
input_seq_data = torch.randn(seq_len, batch_size, input_size)

outputs, final_hidden = model(input_seq_data)
print("Simple RNN Output Shape:", outputs.shape) # (seq_len, batch_size, output_size)
print("Simple RNN Final Hidden State Shape:", final_hidden.shape) # (batch_size, hidden_size)

# 2. PyTorch 内置的 nn.RNN 层
rnn_layer = nn.RNN(input_size, hidden_size, num_layers=1, batch_first=False)

# input: (seq_len, batch_size, input_size)
# h_0: (num_layers * num_directions, batch_size, hidden_size)
h0 = torch.zeros(1, batch_size, hidden_size) # 1层，单向RNN

output, hn = rnn_layer(input_seq_data, h0)
print("\nPyTorch nn.RNN Output Shape:", output.shape)
print("PyTorch nn.RNN Final Hidden State Shape:", hn.shape)

Simple RNN Output Shape: torch.Size([3, 4, 5])
Simple RNN Final Hidden State Shape: torch.Size([4, 20])

PyTorch nn.RNN Output Shape: torch.Size([3, 4, 20])
PyTorch nn.RNN Final Hidden State Shape: torch.Size([1, 4, 20])


### 2.3 局限性：梯度消失与爆炸

尽管基本RNN能够处理序列数据，但它存在严重的局限性：

*   **梯度消失 (Vanishing Gradients):** 随着时间步的增加，梯度在反向传播过程中会呈指数级衰减。这导致模型难以学习到长距离的依赖关系，即较早时间步的信息对当前时间步的影响会变得非常小，类似于“遗忘”。
*   **梯度爆炸 (Exploding Gradients):** 梯度在反向传播过程中可能会呈指数级增长，导致模型权重更新过大，训练不稳定，甚至出现NaN值。这通常可以通过梯度裁剪 (Gradient Clipping) 来缓解。

## 3. RNN 变体

为了克服基本RNN的局限性，特别是梯度消失问题，研究者们提出了多种RNN变体。

### 3.1 深度RNN (Deep RNN)

深度RNN通过堆叠多个RNN层来增加模型的深度，从而提高模型的表达能力。每一层的输出作为下一层的输入，隐藏状态和输出都在层之间传递。

数学表达式变为：
$h_t^{(l)} = f_l(h_t^{(l-1)}, h_{t-1}^{(l)}, x_t)$
其中 $l$ 表示层数。

### 3.2 双向RNN (Bidirectional RNN, Bi-RNN)

在某些任务中，当前时间步的输出不仅依赖于过去的输入，也依赖于未来的输入（例如，在文本处理中，理解一个词的含义可能需要看它后面的词）。双向RNN解决了这个问题。

双向RNN包含两个独立的RNN层：一个从前向后处理序列，另一个从后向前处理序列。最终的隐藏状态是两个方向隐藏状态的拼接（或求和、平均等）。

**数学表达式：**
*   前向隐藏状态：$\vec{h_t} = f_1(\vec{h_{t-1}}, x_t)$
*   后向隐藏状态：$\overleftarrow{h_t} = f_2(\overleftarrow{h_{t+1}}, x_t)$
*   最终隐藏状态：$h_t = [\vec{h_t}; \overleftarrow{h_t}]$ (拼接)
*   最终输出：$y_t = f_3(h_t)$ (PPT中写作 $y = f_3(a, c)$，其中 $a$ 是前向隐藏状态， $c$ 是后向隐藏状态)

In [2]:
import torch.nn as nn

# 深度 RNN (num_layers > 1)
input_size = 10
hidden_size = 20
output_size = 5
seq_len = 3
batch_size = 4
num_layers = 2 # 堆叠2层RNN

deep_rnn_layer = nn.RNN(input_size, hidden_size, num_layers=num_layers, batch_first=False)
h0_deep = torch.zeros(num_layers, batch_size, hidden_size)

output_deep, hn_deep = deep_rnn_layer(input_seq_data, h0_deep)
print("\nPyTorch Deep RNN Output Shape:", output_deep.shape)
print("PyTorch Deep RNN Final Hidden State Shape:", hn_deep.shape) # (num_layers, batch_size, hidden_size)

# 双向 RNN (bidirectional=True)
bi_rnn_layer = nn.RNN(input_size, hidden_size, num_layers=1, bidirectional=True, batch_first=False)
# 对于双向RNN，初始隐藏状态 h0 的 num_layers * num_directions 会变成 1 * 2 = 2
h0_bi = torch.zeros(1 * 2, batch_size, hidden_size)

output_bi, hn_bi = bi_rnn_layer(input_seq_data, h0_bi)
print("\nPyTorch Bidirectional RNN Output Shape:", output_bi.shape) # (seq_len, batch_size, hidden_size * 2)
print("PyTorch Bidirectional RNN Final Hidden State Shape:", hn_bi.shape) # (num_directions, batch_size, hidden_size)


PyTorch Deep RNN Output Shape: torch.Size([3, 4, 20])
PyTorch Deep RNN Final Hidden State Shape: torch.Size([2, 4, 20])

PyTorch Bidirectional RNN Output Shape: torch.Size([3, 4, 40])
PyTorch Bidirectional RNN Final Hidden State Shape: torch.Size([2, 4, 20])


## 4. 长短期记忆网络 (Long Short-Term Memory, LSTM)

LSTM是为了解决基本RNN的梯度消失问题而设计的，它通过引入“门控机制”和“细胞状态”（Cell State）来更有效地管理和记忆信息。

### 4.1 核心思想：细胞状态与门控机制

*   **细胞状态 (Cell State, $c_t$)：** 类似于一条“高速公路”，信息可以直接流过而不受干扰。它负责存储长期记忆。$c_t$ 的变化通常比较缓慢。
*   **隐藏状态 (Hidden State, $h_t$)：** 负责存储短期记忆和当前时间步的输出信息。$h_t$ 的变化可以很快。
*   **门控机制 (Gating Mechanism)：** LSTM通过三个门来控制信息流：
    1.  **遗忘门 (Forget Gate, $z^f$)：** 决定从细胞状态中丢弃哪些信息。
    2.  **输入门 (Input Gate, $z^i$)：** 决定将哪些新信息添加到细胞状态中。
    3.  **输出门 (Output Gate, $z^o$)：** 决定从细胞状态中输出哪些信息到隐藏状态。

每个门都使用 sigmoid 激活函数，输出一个介于0到1之间的数值，表示通过信息的比例。

### 4.2 数学表达式

在时间步 $t$：

1.  **遗忘门 ($z^f$)：**
    $z^f = \sigma(W_f [h_{t-1}, x_t] + b_f)$
    它根据当前输入 $x_t$ 和上一隐藏状态 $h_{t-1}$ 来决定 $c_{t-1}$ 中哪些信息需要被遗忘。

2.  **输入门 ($z^i$) 和候选细胞状态 ($\tilde{c}_t$)：**
    $z^i = \sigma(W_i [h_{t-1}, x_t] + b_i)$
    $\tilde{c}_t = \tanh(W_c [h_{t-1}, x_t] + b_c)$
    输入门 $z^i$ 决定哪些新信息会被添加到细胞状态中，$\tilde{c}_t$ 则是当前输入的候选细胞状态。

3.  **更新细胞状态 ($c_t$)：**
    $c_t = z^f \odot c_{t-1} + z^i \odot \tilde{c}_t$
    旧的细胞状态 $c_{t-1}$ 被遗忘门 $z^f$ 加权，新的候选细胞状态 $\tilde{c}_t$ 被输入门 $z^i$ 加权，然后两者相加，得到新的细胞状态 $c_t$。

    **PPT第十页提到了 "Peephole Connections"**：这是一种LSTM的变体，允许门控层访问细胞状态 $c_{t-1}$。如果启用 Peephole，则门控的计算会变为：
    $z^f = \sigma(W_f [h_{t-1}, x_t, c_{t-1}] + b_f)$
    $z^i = \sigma(W_i [h_{t-1}, x_t, c_{t-1}] + b_i)$
    $z^o = \sigma(W_o [h_{t-1}, x_t, c_{t}] + b_o)$ (注意这里是 $c_t$ 而不是 $c_{t-1}$)

4.  **输出门 ($z^o$)：**
    $z^o = \sigma(W_o [h_{t-1}, x_t] + b_o)$
    输出门 $z^o$ 决定细胞状态 $c_t$ 的哪些部分会被用来计算当前时间步的隐藏状态 $h_t$。

5.  **更新隐藏状态 ($h_t$)：**
    $h_t = z^o \odot \tanh(c_t)$
    新的隐藏状态 $h_t$ 是通过输出门 $z^o$ 对经过 $\tanh$ 激活的细胞状态 $c_t$ 进行加权得到的。

6.  **输出 ($y_t$)：**
    $y_t = \sigma(W_y h_t + b_y)$ (PPT中写作 $y^t = \sigma(W' h^t)$)

### 4.3 LSTM 的优势

LSTM通过门控机制和细胞状态，有效缓解了梯度消失问题，使其能够学习并记忆长距离依赖关系。

### 4.4 LSTM 的实证分析 (A Search Space Odyssey)

这项研究（Greff et al., 2017 "LSTM: A Search Space Odyssey"）对不同LSTM架构进行系统性评估，得出了一些重要结论：
*   **标准LSTM表现良好：** 原始的LSTM架构在多数任务上都能取得非常好的性能。
*   **遗忘门至关重要 (Forget gate is critical for performance)：** 移除遗忘门会显著损害LSTM的性能，这表明遗忘旧信息的能力对于处理序列数据非常重要。
*   **输出门激活函数至关重要 (Output gate activation function is critical)：** 输出门的激活函数（通常是sigmoid）的选择对性能有显著影响。
*   **耦合输入和遗忘门 (Coupling Input and Forget Gate) 和移除窥孔连接 (removing Peepholes) 影响较小：** 这些变体通常不会对性能产生太大影响。

In [3]:
import torch.nn as nn

# PyTorch 内置的 nn.LSTM 层
lstm_layer = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=True, batch_first=False)

# input: (seq_len, batch_size, input_size)
# h_0: (num_layers * num_directions, batch_size, hidden_size)
# c_0: (num_layers * num_directions, batch_size, hidden_size)
h0_lstm = torch.zeros(num_layers * 2, batch_size, hidden_size) # 2层，双向LSTM
c0_lstm = torch.zeros(num_layers * 2, batch_size, hidden_size)

output_lstm, (hn_lstm, cn_lstm) = lstm_layer(input_seq_data, (h0_lstm, c0_lstm))
print("\nPyTorch LSTM Output Shape:", output_lstm.shape)
print("PyTorch LSTM Final Hidden State Shape:", hn_lstm.shape)
print("PyTorch LSTM Final Cell State Shape:", cn_lstm.shape)


PyTorch LSTM Output Shape: torch.Size([3, 4, 40])
PyTorch LSTM Final Hidden State Shape: torch.Size([4, 4, 20])
PyTorch LSTM Final Cell State Shape: torch.Size([4, 4, 20])


## 5. 门控循环单元 (Gated Recurrent Unit, GRU)

GRU是LSTM的简化版本，由Cho等人于2014年提出。它将遗忘门和输入门合并为一个“更新门”（Update Gate），并将细胞状态和隐藏状态合并。这使得GRU的参数更少，计算更简单，但性能通常与LSTM相当。

### 5.1 门控机制

*   **更新门 (Update Gate, $z^t$)：** 决定有多少旧信息需要保留，有多少新信息需要添加。
*   **重置门 (Reset Gate, $r^t$)：** 决定如何将旧的隐藏状态与新的输入结合。

### 5.2 数学表达式

在时间步 $t$：

1.  **更新门 ($z^t$)：**
    $z^t = \sigma(W_z [h_{t-1}, x_t] + b_z)$

2.  **重置门 ($r^t$)：**
    $r^t = \sigma(W_r [h_{t-1}, x_t] + b_r)$

3.  **候选隐藏状态 ($\tilde{h}_t$)：**
    $\tilde{h}_t = \tanh(W_h [r^t \odot h_{t-1}, x_t] + b_h)$
    这里的 $r^t \odot h_{t-1}$ 表示通过重置门来选择性地遗忘前一个隐藏状态的信息。

4.  **更新隐藏状态 ($h_t$)：**
    $h_t = (1 - z^t) \odot h_{t-1} + z^t \odot \tilde{h}_t$
    更新门 $z^t$ 决定了最终隐藏状态有多少来自旧隐藏状态，有多少来自新的候选隐藏状态。

### 5.3 GRU 的优势

*   参数更少，计算效率更高。
*   在许多任务上性能与LSTM相当，有时甚至更好。

In [4]:
import torch.nn as nn

# PyTorch 内置的 nn.GRU 层
gru_layer = nn.GRU(input_size, hidden_size, num_layers=num_layers, bidirectional=True, batch_first=False)

# input: (seq_len, batch_size, input_size)
# h_0: (num_layers * num_directions, batch_size, hidden_size)
h0_gru = torch.zeros(num_layers * 2, batch_size, hidden_size)

output_gru, hn_gru = gru_layer(input_seq_data, h0_gru)
print("\nPyTorch GRU Output Shape:", output_gru.shape)
print("PyTorch GRU Final Hidden State Shape:", hn_gru.shape)


PyTorch GRU Output Shape: torch.Size([3, 4, 40])
PyTorch GRU Final Hidden State Shape: torch.Size([4, 4, 20])


## 6. 嵌套LSTM (Nested LSTM)

嵌套LSTM（Moniz et al., 2018）是一种更复杂的LSTM变体，旨在捕获多层次的时间依赖性。它通过在传统LSTM单元内嵌入额外的LSTM单元，形成层级结构。
*   **(a) LSTM:** 传统LSTM单元。
*   **(b) Stacked LSTM:** 堆叠的LSTM层，层与层之间是线性传递。
*   **(c) Nested LSTM:** 更复杂的依赖关系，内部LSTM单元的状态影响外部单元的状态，形成一种“层次化”的记忆。这可以用于处理具有复杂时间结构的任务。

## 7. RNN在自然语言处理 (NLP) 中的应用

NLP任务本质上就是序列数据处理。RNN及其变体在NLP领域取得了巨大成功，尤其是在长距离依赖问题上。

### 7.1 长距离依赖 (Long-distance Dependencies)

文本中经常存在相距较远的词语之间的语义或语法联系，这种现象被称为长距离依赖。

1.  **数、性别一致性 (Agreement in number, gender, etc.)：**
    *   “He does not have very much confidence in **himself**.” (He 和 himself 的指代一致)
    *   “She does not have very much confidence in **herself**.” (She 和 herself 的指代一致)
2.  **选择偏好 (Selectional preference)：**
    *   “The **reign** has lasted as long as the life of the **queen**.” (reign 与 queen 相关)
    *   “The **rain** has lasted as long as the life of the **clouds**.” (rain 与 clouds 相关)
3.  **指代消解 (Referent of "it")：**
    *   “The trophy would not fit in the brown suitcase because **it** was too **big**.” (it 指 suitcase)
    *   “The trophy would not fit in the brown suitcase because **it** was too **small**.” (it 指 trophy)

基本RNN难以捕捉这些长距离依赖，而LSTM和GRU通过其门控机制，能够更有效地保留和传递远距离信息。

### 7.2 时间展开 (Unrolling in Time)

将RNN在时间维度上“展开”，可以看到每个时间步的输入、隐藏状态和输出。这有助于理解RNN如何处理序列。
例如，情感分析任务中，输入句子“I hate this movie”，RNN在每个时间步处理一个词，最终在序列末尾进行预测。

### 7.3 RNN 的应用场景

*   **表示句子 (Represent a sentence)：** RNN可以通过其最终隐藏状态或所有时间步的隐藏状态来捕获整个句子的语义信息，用于句子分类、检索等。
*   **表示上下文 (Represent a context within a sentence)：** 在每个时间步，RNN的隐藏状态都包含了到目前为止的上下文信息。这对于序列标注（如词性标注、命名实体识别）、语言建模等任务非常有用。
*   **序列到序列任务 (Sequence-to-Sequence, Seq2Seq)：**

    Seq2Seq模型由一个编码器（Encoder）和一个解码器（Decoder）组成，通常都使用RNN（或LSTM/GRU）。
    *   **编码器：** 读取输入序列（如源语言句子），并将其编码成一个固定长度的上下文向量（通常是编码器最终的隐藏状态）。
    *   **解码器：** 接收编码器生成的上下文向量，并逐步生成输出序列（如目标语言句子）。
    主要应用于机器翻译、文本摘要、对话系统等。


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

print(f"PyTorch Version: {torch.__version__}")

# --- 1. 基本RNN的前向传播 (手动实现简化版) ---
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        
        # 定义权重矩阵和偏置
        self.W_xh = nn.Linear(input_size, hidden_size, bias=True) # W_i, bias_i
        self.W_hh = nn.Linear(hidden_size, hidden_size, bias=True) # W_h, bias_h
        self.W_hy = nn.Linear(hidden_size, output_size, bias=True) # W_o, bias_o
        
        self.activation = nn.Tanh() # 激活函数，PPT中用sigma表示

    def forward(self, input_seq):
        # input_seq: (seq_len, batch_size, input_size)
        
        # 初始化隐藏状态 h0
        # h0: (batch_size, hidden_size)
        batch_size = input_seq.size(1)
        # 确保 hidden_state 在与 input_seq 相同的设备上
        hidden_state = torch.zeros(batch_size, self.hidden_size, device=input_seq.device)
        
        outputs = []
        for i in range(input_seq.size(0)): # 遍历序列中的每个时间步
            current_input = input_seq[i] # current_input: (batch_size, input_size)
            
            # 计算新的隐藏状态
            # h_t = activation(W_xh * x_t + W_hh * h_{t-1} + b_h)
            combined_input = self.W_xh(current_input) + self.W_hh(hidden_state)
            hidden_state = self.activation(combined_input)
            
            # 计算当前输出
            # y_t = activation(W_hy * h_t + b_y)
            output = self.W_hy(hidden_state)
            outputs.append(output)
            
        # outputs: list of (batch_size, output_size) -> concatenate to (seq_len, batch_size, output_size)
        return torch.stack(outputs, dim=0), hidden_state

# 示例使用
input_size = 10
hidden_size = 20
output_size = 5
seq_len = 3
batch_size = 4 # 保持 batch_size > 1 以测试
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleRNN(input_size, hidden_size, output_size).to(device)
input_seq_data = torch.randn(seq_len, batch_size, input_size).to(device)

outputs, final_hidden = model(input_seq_data)
print("--- Simple RNN ---")
print("Simple RNN Output Shape:", outputs.shape) # (seq_len, batch_size, output_size)
print("Simple RNN Final Hidden State Shape:", final_hidden.shape) # (batch_size, hidden_size)

# --- 2. PyTorch 内置的 nn.RNN 层 ---
rnn_layer = nn.RNN(input_size, hidden_size, num_layers=1, batch_first=False).to(device)

# input: (seq_len, batch_size, input_size)
# h_0: (num_layers * num_directions, batch_size, hidden_size)
h0 = torch.zeros(1, batch_size, hidden_size).to(device) # 1层，单向RNN

output, hn = rnn_layer(input_seq_data, h0)
print("\n--- PyTorch nn.RNN Layer ---")
print("PyTorch nn.RNN Output Shape:", output.shape)
print("PyTorch nn.RNN Final Hidden State Shape:", hn.shape)

# --- 3. 深度 RNN (num_layers > 1) ---
num_layers = 2 # 堆叠2层RNN

deep_rnn_layer = nn.RNN(input_size, hidden_size, num_layers=num_layers, batch_first=False).to(device)
h0_deep = torch.zeros(num_layers, batch_size, hidden_size).to(device)

output_deep, hn_deep = deep_rnn_layer(input_seq_data, h0_deep)
print("\n--- PyTorch Deep RNN ---")
print("PyTorch Deep RNN Output Shape:", output_deep.shape)
print("PyTorch Deep RNN Final Hidden State Shape:", hn_deep.shape) # (num_layers, batch_size, hidden_size)

# --- 4. 双向 RNN (bidirectional=True) ---
bi_rnn_layer = nn.RNN(input_size, hidden_size, num_layers=1, bidirectional=True, batch_first=False).to(device)
# 对于双向RNN，初始隐藏状态 h0 的 num_layers * num_directions 会变成 1 * 2 = 2
h0_bi = torch.zeros(1 * 2, batch_size, hidden_size).to(device)

output_bi, hn_bi = bi_rnn_layer(input_seq_data, h0_bi)
print("\n--- PyTorch Bidirectional RNN ---")
print("PyTorch Bidirectional RNN Output Shape:", output_bi.shape) # (seq_len, batch_size, hidden_size * 2)
print("PyTorch Bidirectional RNN Final Hidden State Shape:", hn_bi.shape) # (num_directions, batch_size, hidden_size)

# --- 5. PyTorch 内置的 nn.LSTM 层 ---
lstm_layer = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=True, batch_first=False).to(device)

# input: (seq_len, batch_size, input_size)
# h_0: (num_layers * num_directions, batch_size, hidden_size)
# c_0: (num_layers * num_directions, batch_size, hidden_size)
h0_lstm = torch.zeros(num_layers * 2, batch_size, hidden_size).to(device) # 2层，双向LSTM
c0_lstm = torch.zeros(num_layers * 2, batch_size, hidden_size).to(device)

output_lstm, (hn_lstm, cn_lstm) = lstm_layer(input_seq_data, (h0_lstm, c0_lstm))
print("\n--- PyTorch LSTM Layer ---")
print("PyTorch LSTM Output Shape:", output_lstm.shape)
print("PyTorch LSTM Final Hidden State Shape:", hn_lstm.shape)
print("PyTorch LSTM Final Cell State Shape:", cn_lstm.shape)

# --- 6. PyTorch 内置的 nn.GRU 层 ---
gru_layer = nn.GRU(input_size, hidden_size, num_layers=num_layers, bidirectional=True, batch_first=False).to(device)

# input: (seq_len, batch_size, input_size)
# h_0: (num_layers * num_directions, batch_size, hidden_size)
h0_gru = torch.zeros(num_layers * 2, batch_size, hidden_size).to(device)

output_gru, hn_gru = gru_layer(input_seq_data, h0_gru)
print("\n--- PyTorch GRU Layer ---")
print("PyTorch GRU Output Shape:", output_gru.shape)
print("PyTorch GRU Final Hidden State Shape:", hn_gru.shape)


# --- 7. Encoder-Decoder 模型的概念结构 (LSTM) ---
# 假设词嵌入维度为 embed_dim
# 编码器 (Encoder)
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embed_dim, hidden_size, num_layers=1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embed_dim) # 词嵌入层
        # batch_first=False 是 nn.LSTM 的默认值，表示输入形状是 (seq_len, batch_size, input_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_size, num_layers, batch_first=False) 

    def forward(self, input_seq):
        # input_seq: (seq_len, batch_size) - 包含词汇表索引的张量
        embedded = self.embedding(input_seq) # (seq_len, batch_size, embed_dim)
        
        # 初始化编码器的 h0 和 c0 为全零张量
        # 形状为 (num_layers * num_directions, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, input_seq.size(1), self.hidden_size, device=input_seq.device)
        c0 = torch.zeros(self.num_layers, input_seq.size(1), self.hidden_size, device=input_seq.device)

        # output: 所有时间步的输出 (seq_len, batch_size, hidden_size)
        # hidden, cell: 最终隐藏状态和细胞状态 (num_layers, batch_size, hidden_size)
        output, (hidden, cell) = self.rnn(embedded, (h0, c0)) 
        return hidden, cell # 编码器通常只返回最终的隐藏状态和细胞状态作为上下文向量

# 解码器 (Decoder)
class DecoderRNN(nn.Module):
    def __init__(self, output_size, embed_dim, hidden_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(output_size, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_size, num_layers, batch_first=False)
        self.out = nn.Linear(hidden_size, output_size) # 输出层，将隐藏状态映射到词汇表大小

    def forward(self, input_step, hidden, cell):
        # input_step: (1, batch_size) - 单个时间步的输入 (如 <SOS> 或前一个预测的词索引)
        # hidden, cell: 编码器最终的隐藏状态和细胞状态，作为解码器的初始状态
        
        embedded = self.embedding(input_step) # (1, batch_size, embed_dim)
        
        # 传递隐藏状态和细胞状态给 LSTM
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell)) 
        
        # output: (1, batch_size, hidden_size) -> prediction: (batch_size, output_size)
        # squeeze(0) 将序列长度的维度移除
        prediction = self.out(output.squeeze(0)) 
        return prediction, hidden, cell

# 示例使用
input_vocab_size = 1000 # 输入词汇表大小
output_vocab_size = 500 # 输出词汇表大小
embed_dim = 256
hidden_size = 512
num_layers_enc_dec = 1 # 编码器和解码器都使用1层LSTM，以确保 hidden/cell 维度匹配

encoder = EncoderRNN(input_vocab_size, embed_dim, hidden_size, num_layers=num_layers_enc_dec).to(device)
decoder = DecoderRNN(output_vocab_size, embed_dim, hidden_size, num_layers=num_layers_enc_dec).to(device)

# 模拟编码过程
src_seq_len = 10
batch_size = 4 # 保持 batch_size > 1
encoder_input = torch.randint(0, input_vocab_size, (src_seq_len, batch_size)).to(device) # 随机生成输入序列

encoder_hidden, encoder_cell = encoder(encoder_input)
print("\n--- Encoder-Decoder (LSTM) ---")
print("Encoder final hidden state shape:", encoder_hidden.shape)
print("Encoder final cell state shape:", encoder_cell.shape)

# 模拟解码过程的第一个时间步
# Decoder的第一个输入通常是特殊符号 <SOS> (Start Of Sequence)，假设 0 是 <SOS> 的索引
# 关键修正：确保 decoder_input 形状为 (seq_len=1, batch_size) 且数据类型为 long
decoder_input = torch.full((1, batch_size), 0, dtype=torch.long).to(device)

decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, encoder_hidden, encoder_cell)
print("Decoder first step output shape:", decoder_output.shape) # (batch_size, output_vocab_size)
print("Decoder first step hidden state shape:", decoder_hidden.shape)
print("Decoder first step cell state shape:", decoder_cell.shape)


# --- 8. 一个简单的点积注意力机制 ---
class DotProductAttention(nn.Module):
    def __init__(self, hidden_size):
        super(DotProductAttention, self).__init__()
        # Query, Key, Value 都是 hidden_size
        # 为了更通用，即使点积注意力通常不包含额外的线性层，这里为了与多头注意力概念统一，依然保留
        # 在实际的Transformer中，Query/Key/Value 的线性层是独立的
        self.query_transform = nn.Linear(hidden_size, hidden_size, bias=False)
        self.key_transform = nn.Linear(hidden_size, hidden_size, bias=False)
        self.value_transform = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, query, keys, values, mask=None):
        # query: (1, batch_size, hidden_size) - 通常是解码器当前时间步的隐藏状态
        # keys: (seq_len, batch_size, hidden_size) - 编码器所有时间步的输出
        # values: (seq_len, batch_size, hidden_size) - 通常与keys相同

        # 变换 Q, K, V
        # Q: (batch_size, hidden_size)
        q = self.query_transform(query.squeeze(0)) 
        
        # K: (batch_size, hidden_size, seq_len)
        k = self.key_transform(keys).permute(1, 2, 0) 
        
        # V: (batch_size, seq_len, hidden_size)
        v = self.value_transform(values).permute(1, 0, 2) 

        # 计算注意力分数 (点积)
        # (batch_size, 1, hidden_size) @ (batch_size, hidden_size, seq_len) -> (batch_size, 1, seq_len)
        scores = torch.bmm(q.unsqueeze(1), k).squeeze(1) 
        
        # 缩放 (Scaled Dot Product Attention 的一部分)
        # 这里使用 keys 的最后一个维度作为 dk
        dk = keys.size(-1) 
        scores = scores / (dk ** 0.5) 

        # 应用mask (可选，用于避免关注padding等)
        if mask is not None:
            # mask 的形状应为 (batch_size, seq_len)
            scores = scores.masked_fill(mask == 0, -torch.inf) # 使用 -torch.inf 或 -1e9

        # 归一化注意力权重
        attention_weights = F.softmax(scores, dim=-1) # (batch_size, seq_len)

        # 计算上下文向量
        # (batch_size, 1, seq_len) @ (batch_size, seq_len, hidden_size) -> (batch_size, 1, hidden_size)
        context_vector = torch.bmm(attention_weights.unsqueeze(1), v).squeeze(1) # (batch_size, hidden_size)

        return context_vector, attention_weights

# 示例使用
# 模拟解码器当前隐藏状态 (query)
query_hidden = torch.randn(1, batch_size, hidden_size).to(device) 
# 模拟编码器所有时间步的输出 (keys 和 values)
encoder_outputs_attn = torch.randn(seq_len, batch_size, hidden_size).to(device) 

attention_model = DotProductAttention(hidden_size).to(device)
context, attn_weights = attention_model(query_hidden, encoder_outputs_attn, encoder_outputs_attn)

print("\n--- Dot Product Attention ---")
print("Attention Context Vector Shape:", context.shape) # (batch_size, hidden_size)
print("Attention Weights Shape:", attn_weights.shape) # (batch_size, seq_len)

PyTorch Version: 2.5.1
--- Simple RNN ---
Simple RNN Output Shape: torch.Size([3, 4, 5])
Simple RNN Final Hidden State Shape: torch.Size([4, 20])

--- PyTorch nn.RNN Layer ---
PyTorch nn.RNN Output Shape: torch.Size([3, 4, 20])
PyTorch nn.RNN Final Hidden State Shape: torch.Size([1, 4, 20])

--- PyTorch Deep RNN ---
PyTorch Deep RNN Output Shape: torch.Size([3, 4, 20])
PyTorch Deep RNN Final Hidden State Shape: torch.Size([2, 4, 20])

--- PyTorch Bidirectional RNN ---
PyTorch Bidirectional RNN Output Shape: torch.Size([3, 4, 40])
PyTorch Bidirectional RNN Final Hidden State Shape: torch.Size([2, 4, 20])

--- PyTorch LSTM Layer ---
PyTorch LSTM Output Shape: torch.Size([3, 4, 40])
PyTorch LSTM Final Hidden State Shape: torch.Size([4, 4, 20])
PyTorch LSTM Final Cell State Shape: torch.Size([4, 4, 20])

--- PyTorch GRU Layer ---
PyTorch GRU Output Shape: torch.Size([3, 4, 40])
PyTorch GRU Final Hidden State Shape: torch.Size([4, 4, 20])

--- Encoder-Decoder (LSTM) ---
Encoder final hidden

## 8. 注意力机制 (Attention Mechanism)

虽然Seq2Seq模型在机器翻译等任务中取得了成功，但编码器将整个输入序列压缩成一个固定长度的上下文向量，对于长序列来说，这会造成信息瓶颈，导致模型难以处理。注意力机制（Attention Mechanism）的引入解决了这个问题。

### 8.1 引入动机：模拟人类注意力

人类在处理信息时，会根据任务需求将注意力集中在输入序列中最重要的部分。注意力机制旨在模仿这种行为，允许模型在生成输出时，动态地“关注”输入序列中的相关部分。

### 8.2 核心思想：Query, Key, Value

注意力机制的核心是计算“查询”（Query）与“键”（Key）之间的相似度，然后将这些相似度作为权重，对“值”（Value）进行加权求和，得到最终的上下文向量。
*   **Query (Q):** 通常是解码器当前时间步的隐藏状态，代表当前要关注什么。
*   **Key (K):** 编码器所有时间步的隐藏状态（或部分），代表可能被关注的信息点。
*   **Value (V):** 编码器所有时间步的隐藏状态（或部分），即实际要提取的信息。

在最常见的Encoder-Decoder Attention中，Key和Value通常是编码器所有时间步的隐藏状态，而Query是解码器当前时间步的隐藏状态。

### 8.3 计算过程

1.  **计算注意力得分 (Attention Score)：** 计算 Query 与每个 Key 之间的相似度。常用的相似度函数有：
    *   **点积 (Dot Product):** $score(Q, K) = Q^T K$
    *   **缩放点积 (Scaled Dot Product):** $score(Q, K) = \frac{Q^T K}{\sqrt{d_k}}$ (PPT第三十页重点介绍)
    *   **加性注意力/MLP注意力 (Additive/MLP Attention):** $score(Q, K) = v^T \tanh(W_1 Q + W_2 K)$
    *   **双线性注意力 (Bilinear Attention):** $score(Q, K) = Q^T W K$ (PPT第三十页)

2.  **归一化注意力权重 (Normalize Attention Weights)：** 使用 Softmax 函数对注意力得分进行归一化，得到注意力权重 (Attention Weights)。这些权重是介于0到1之间的概率分布，总和为1。
    $attention\_weights = \text{softmax}(score(Q, K_i))$

3.  **计算上下文向量 (Context Vector)：** 将注意力权重与对应的 Value 进行加权求和，得到上下文向量。
    $context\_vector = \sum_i attention\_weights_i \times V_i$

这个上下文向量会被送入解码器，帮助其更准确地生成当前时间步的输出。

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DotProductAttention(nn.Module):
    def __init__(self, hidden_size):
        super(DotProductAttention, self).__init__()
        # Query, Key, Value 都是 hidden_size
        self.query_transform = nn.Linear(hidden_size, hidden_size, bias=False)
        self.key_transform = nn.Linear(hidden_size, hidden_size, bias=False)
        self.value_transform = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, query, keys, values, mask=None):
        # query: (1, batch_size, hidden_size) or (batch_size, hidden_size) for single step
        # keys: (seq_len, batch_size, hidden_size) - 编码器所有时间步的输出
        # values: (seq_len, batch_size, hidden_size) - 通常与keys相同

        # 变换 Q, K, V (可选，但通常会进行线性变换)
        q = self.query_transform(query.squeeze(0)) # (batch_size, hidden_size)
        k = self.key_transform(keys).permute(1, 2, 0) # (batch_size, hidden_size, seq_len)
        v = self.value_transform(values).permute(1, 0, 2) # (batch_size, seq_len, hidden_size)

        # 计算注意力分数 (点积)
        # (batch_size, hidden_size) @ (batch_size, hidden_size, seq_len) -> (batch_size, seq_len)
        scores = torch.bmm(q.unsqueeze(1), k).squeeze(1) 
        
        # 缩放 (Scaled Dot Product Attention 的一部分)
        # scores = scores / (keys.size(-1) ** 0.5) # 如果是Scaled Dot Product Attention，则需要除以sqrt(d_k)

        # 应用mask (可选，用于避免关注padding等)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9) # mask == 0 的地方设为负无穷

        # 归一化注意力权重
        attention_weights = F.softmax(scores, dim=-1) # (batch_size, seq_len)

        # 计算上下文向量
        # (batch_size, 1, seq_len) @ (batch_size, seq_len, hidden_size) -> (batch_size, 1, hidden_size)
        context_vector = torch.bmm(attention_weights.unsqueeze(1), v).squeeze(1) # (batch_size, hidden_size)

        return context_vector, attention_weights

# 示例使用
query_hidden = torch.randn(1, batch_size, hidden_size) # 模拟解码器当前隐藏状态
encoder_outputs = torch.randn(seq_len, batch_size, hidden_size) # 模拟编码器所有时间步的输出

attention_model = DotProductAttention(hidden_size)
context, attn_weights = attention_model(query_hidden, encoder_outputs, encoder_outputs)

print("\nAttention Context Vector Shape:", context.shape) # (batch_size, hidden_size)
print("Attention Weights Shape:", attn_weights.shape) # (batch_size, seq_len)


Attention Context Vector Shape: torch.Size([4, 512])
Attention Weights Shape: torch.Size([4, 3])


### 8.4 注意力得分函数 (Attention Score Functions)

*   **多层感知机注意力 (Multi-layer Perceptron, MLP Attention / Additive Attention):** 
    $a(q, k) = v_a^T \tanh(W_q q + W_k k)$
    优点：灵活，在大型数据集上表现良好。
*   **双线性注意力 (Bilinear Attention):**
    $a(q, k) = q^T W k$
    它不引入额外的非线性，通过一个矩阵 $W$ 将 Query 和 Key 连接起来。
*   **点积注意力 (Dot Product Attention):**
    $a(q, k) = q^T k$
    优点：简单，无额外参数。
    缺点：当维度 $d_k$ 很大时，点积结果容易很大，导致 Softmax 梯度过小。
*   **缩放点积注意力 (Scaled Dot Product Attention):**
    $a(q, k) = \frac{q^T k}{\sqrt{d_k}}$
    优点：解决了点积注意力在高维时梯度消失的问题，是Transformer中使用的主要形式。

### 8.5 注意力变体 (Attention Variants)

*   **复制机制 (Copying mechanism):** 在生成输出时，除了从预定义词汇表中选择，还可以直接从输入序列中复制单词。
*   **关注前置词 (Attend to previous words):** 在生成输出时，不仅关注输入序列，也关注已经生成的输出序列。
*   **关注多模态输入 (Attend to multi-modal inputs):** 注意力可以应用于图像、语音等不同模态的输入，实现跨模态的关联。
*   **关注多个来源 (Attend to multiple sources):** 在解码时，同时关注多个编码器（例如，处理多语种输入）。

### 8.6 分层注意力 (Hierarchical Structures / Hierarchical Attention)

对于长文档（包含多个句子），可以直接使用Seq2Seq模型处理会面临计算和记忆的挑战。分层注意力提出了一种解决方案：
1.  **词级别注意力 (Word-level Attention)：** 首先对每个句子内部的词进行注意力计算，生成句子表示。
2.  **句子级别注意力 (Sentence-level Attention)：** 然后对这些句子表示进行注意力计算，生成文档表示。
这种结构更符合人类阅读理解长文的习惯，也能够更好地捕获不同粒度的信息。

### 8.7 硬注意力与软注意力 (Hard Attention vs. Soft Attention)

*   **软注意力 (Soft Attention)：** 权重是连续的，通过 Softmax 计算得到，模型在所有输入位置上都分配非零权重。这种方式是可微的，可以直接通过梯度下降训练。
*   **硬注意力 (Hard Attention)：** 权重是离散的（通常是0或1），模型只关注输入序列中的一个或几个特定位置。这种方式不可微，通常需要强化学习等方法进行训练。硬注意力更具可解释性，但训练难度更大。

## 9. Transformer：基于注意力的模型

“Attention Is All You Need”这篇论文（Vaswani et al., 2017）提出了一种完全基于注意力机制的架构——Transformer，彻底改变了序列建模领域。它摒弃了传统的循环（RNN）和卷积（CNN）结构，实现了并行化，并显著提升了长距离依赖的建模能力。

### 9.1 核心组件

*   **多头注意力 (Multi-headed Attention):**
    **PPT第三十五页详细说明：** 它将 Query、Key、Value 分别投影到多个不同的子空间，然后在每个子空间中并行计算注意力。最后将所有子空间的注意力结果拼接起来。这使得模型能够从不同角度和不同表示子空间中捕获信息。
    $MultiHead(Q, K, V) = Concat(head_1, \dots, head_h) W^O$
    其中 $head_i = Attention(QW_i^Q, KW_i^K, VW_i^V)$，这里的 $Attention$ 通常是缩放点积注意力。

*   **自注意力 (Self-Attention / Intra-Attention):**
    **PPT第三十六页详细说明：** 当 Query、Key 和 Value 都来自于同一个输入序列时，注意力机制被称为自注意力。它允许模型在编码单个序列时，计算序列内部不同位置之间的关联性，从而更好地理解每个词在句子中的上下文。例如，在理解“它”指代什么时，自注意力可以关注到句子中相关的名词。

### 9.2 Transformer 架构

Transformer由编码器和解码器组成，每个部分都堆叠了多个相同的层。
*   **编码器：** 包含多头自注意力层和前馈神经网络层，用于将输入序列映射到连续的表示。
*   **解码器：** 包含多头自注意力层、多头编码器-解码器注意力层（Query来自解码器，Key/Value来自编码器），以及前馈神经网络层。解码器还引入了掩码机制，确保在预测当前位置时只能看到之前的位置。

### 9.3 Transformer 的优势

*   **并行化：** 相比RNN的顺序处理，Transformer可以并行计算所有时间步，大大提高了训练效率。
*   **长距离依赖：** 通过自注意力机制，每个位置可以直接计算与序列中任何其他位置的关联，捕获长距离依赖的能力更强。
*   **表达能力：** 多头注意力机制使得模型能够学习到更丰富的表示。

*   **Encoder-Decoder Attention:** 解码器的 Query 与编码器的 Key/Value 交互，用于获取源序列信息。
*   **Self-attention layers in the encoder:** 编码器内部的自注意力，用于理解输入序列内部的关联。
*   **Self-attention layers in the decoder:** 解码器内部的自注意力，用于理解已生成输出序列内部的关联。

## 10. 指针网络 (Pointer Network)

指针网络（Pointer Network，Vinyals et al., 2015）是Seq2Seq模型的一种变体，专门用于解决输出序列的元素是输入序列中元素的索引的问题。

### 10.1 问题背景：Seq2Seq 的局限性

传统的Seq2Seq模型在解码时，通常从一个预定义的、固定大小的词汇表中选择输出。然而，对于某些问题，输出不是一个词汇表中的词，而是输入序列中的某个元素的“位置”或“索引”。
*   **凸包问题 (Convex Hull):** 给定一组二维点，输出构成凸包的点，这些点必须是输入点集中的一部分。
*   **旅行商问题 (Traveling Salesperson Problem, TSP):** 给定一组城市，找到访问所有城市的最短路径，输出是城市访问的顺序（即输入城市的排列）。
*   **文本摘要 (Text Summarization) 和机器翻译 (Machine Translation) 中的复制机制 (Copying Mechanism):** 遇到输入中存在的罕见词（OOV, Out-Of-Vocabulary），传统Seq2Seq难以生成，而直接从输入中复制是更好的选择。

### 10.2 工作原理

指针网络的核心思想是：在解码器的每个时间步，不是预测一个词汇表中的词，而是预测输入序列中的一个**索引**。这个预测是通过注意力机制实现的：将注意力权重直接作为输出概率。

1.  **注意力计算：** 解码器的隐藏状态（Query）与编码器所有时间步的隐藏状态（Key）计算注意力分数。
2.  **概率分布：** 将这些注意力分数经过 Softmax 归一化，得到的注意力权重（通常是 $\alpha_t$）就是输入序列中每个位置被“指向”的概率。
3.  **输出索引：** 选择概率最大的位置作为当前时间步的输出索引。

输出不再是固定的词汇表，而是输入点集的索引 $x_1, x_2, \dots, x_N$。

### 10.3 应用

*   **凸包问题：** 从输入点集中选出形成凸包的点。
*   **旅行商问题：** 确定访问城市的顺序。
*   **抽象摘要/机器翻译中的复制机制：**
    **PPT第四十二页展示了在文本摘要和机器翻译中“复制”的功能：** 对于输入中出现但可能不在模型词汇表中的词（如人名“Guillaume”、“Cesar”或地名“Lausanne”），模型可以通过指针机制直接从源文本中复制过来，而不是试图生成一个词汇表中的词。

## 11. 当代语言模型：ELMo, GPT, BERT

在RNN和注意力机制的推动下，预训练语言模型（Pre-trained Language Models）成为NLP领域的新范式，带来了巨大的突破。
*   **ELMo (Embeddings from Language Models):** 使用双向LSTM在大规模语料上进行预训练，生成上下文相关的词向量。
*   **GPT (Generative Pre-trained Transformer):** 基于Transformer的解码器进行预训练，主要用于生成任务。
*   **BERT (Bidirectional Encoder Representations from Transformers):** 基于Transformer的编码器进行预训练，通过掩码语言模型（Masked Language Model）和下一句预测（Next Sentence Prediction）任务学习双向上下文信息，可用于各种下游任务。

这些模型通过在大规模无标注文本上进行预训练，学习到丰富的语言知识和上下文信息，然后可以通过微调（fine-tuning）在特定任务上取得SOTA（State-Of-The-Art）表现，极大地推动了NLP的发展。这被PPT形象地比喻为“NLP领域的ImageNet时刻”。